# Secondary Data
We are a market research team for a company which specializes in luxury notebooks. The company is interested in developing a new smart pen to pair with their notebooks. Our task is to assess the market demand for such a product, starting with
exploring secondary data. 

## Filtering to find competitors
**Task 1:** Write a function `filter_products_raw` that takes one argument (`key_phrase`) and returns a list of strings; these strings should be all the lines in the `/data/Assignment-2/products.json` file which contain the specified key phrase.

In [1]:
## Hi, please note that this was done on our local systems and some links to the following may be broken 
## a. reading files 
## b. the part in R - we could not install the rpy2 package so we took the file from python, read it in R and wrote codes
## thank you

import demjson
import json
import re
import numpy as np

In [2]:
## A json file of metadata (descriptions, price, salesrank, brand info, and co-purchasing links) for over 600K 
## office and electronic products listed on Amazon. Source & file description: htp://jmcauley.ucsd.edu/data/amazon/
with open('/data/Assignment-2/products.json','r') as f:
        products_raw = f.readlines()

In [3]:
## get to know the data
type(products_raw)
type(products_raw[0])
len(products_raw)
products_raw[2]

"{'asin': '0439886341', 'categories': [['Electronics', 'Computers & Accessories', 'PDAs, Handhelds & Accessories', 'PDAs & Handhelds']], 'description': 'Digital Organizer and Messenger', 'title': 'Digital Organizer and Messenger', 'price': 8.15, 'salesRank': {'Electronics': 144944}, 'imUrl': 'http://ecx.images-amazon.com/images/I/51k0qa8fNML._SX300_.jpg', 'related': {'also_viewed': ['0545016266', 'B009ECM8QY', 'B001CODA9G', 'B000N2X2GO', 'B00005T3UK', 'B00009MVFL', 'B000VU0CQG', 'B0000796XL']}}\n"

In [3]:
#return a list of string
def filter_products_raw(list1,key_phrase):
    pattern=re.compile(r'\b'+key_phrase)
    #pattern=re.compile(r'asin': '(r'\b'+key_phrase)','')
    match=[pattern.findall(t) for t in list1]
    result=[list1[u] for u,v in enumerate(match) if len(v)>0]
    return (result)

**Task 2:** Using the function I just wrote, filtering the products in `products.json` by different key words or phrases.  Print out the total number of products that match the key phrase.    
Also print out the first three products that match the key phrase.   
Then use this technique to find a collection of products that would be in competition with the product our team is exploring.

In [4]:
# filtering the products in products.json by different key words or phrases
matchresult=filter_products_raw(filter_products_raw(products_raw,r'pen\b'),r'smart\b')
# Print out the total number of products that match the key phrase
ttl_match=len(matchresult)
ttl_match

422

In [6]:
#a collection of products that would be in competition with the product your team is exploring
#because there are a lot of products are case, so we filter that our
casepattern=re.compile(r'\b'+r'case\b')
filterd=list(filter(lambda x: not casepattern.search(x),matchresult))
#okay_items = [x for x in matchresult if not casepattern.match(x)]
len(filterd)

130

In [7]:
#print out the first three products that match the key phrase
firstthree=filterd[:3]
print(firstthree)

['{\'asin\': \'B00002EQCF\', \'description\': \'The new 3Com Palm VII connected organizer is just about everything you\\\'d want in a pocket computer: It\\\'s useful, easy to operate and wirelessly connected. But the price of the connectivity is far beyond what most road warriors would be willing to pay.The Palm VII is essentially a Palm III with wireless connectivity built in. While there are plenty of wireless solutions for palm-sized computers, the Palm VII\\\'s is by far the most elegant and the easiest to set up and use.This latest version of the Palm is not the most advanced 3Com device in all respects. It offers neither the Palm IIIx\\\'s superior operating system and memory, nor the Palm V\\\'s stylish svelteness and rechargeable battery. The battery life is half that of the Palm III-the Palm VII will runs through two AAA batteries in about two or three weeks. It\\\'s also significantly bigger and heavier -- and in my opinion, uglier -- than any other Palm, making it too big fo

**Task 3:** With the `filter_products_raw` function as a starting point, create a new function `filter_products_df` that returns a pandas data frame with two columns: `product.id` and `price`.  

In [8]:
import pandas as pd

In [9]:
#return  pandas data frame with two columns: product.id and price.
def filter_products_df(list2):
    regex1 = re.compile(r"'asin': '(.+?)',")
    regex2 = re.compile(r"'price': (.+?),")
    pid=[regex1.findall(t) for t in filterd]
    price=[regex2.findall(t) for t in filterd]
    df=pd.DataFrame({"product.id":pid,"price":price})
    result=(df[df.price.apply(len)>0]).applymap(lambda x: x[0])
    return (result. reset_index(drop=True))

**Task 4:** Using the new `filter_products_df` function, filter the products by the key phrase identified in Task 2, resulting in a data frame of product ids with their corresponding prices; call this data frame variable `products`.

In [11]:
#using the function
products=filter_products_df(filterd)
products.head()

,product.id,price
0,B00002EQCF,99.0
1,B000F4CUKY,117.57
2,B0013T1Z7O,59.99
3,B001DG1DNI,25.99
4,B001NEK0P2,49.89


In [12]:
products.describe()

,product.id,price
count,107,107
unique,107,68
top,B008YWGXE8,13.99
freq,1,6


## Filtering competitor reviews
**Task 6:** Write a function called `filter_reviews` to help find all the reviews that match the products which match the key phrase query.  This function take a `products` argument and return a data frame with one review per row and three columns: `product.id`, `rating` (on the 5-star scale), and `review` (text).

In [15]:
from demjson import decode

In [16]:
def filter_reviews(products):
    df = pd.DataFrame({"product.id": [], "rating": [], "review": []})
    f = open("/data/Assignment-2/reviews.json")
    prods = set(products["product.id"])
    for line in f:
        id = line.split(':')[2].split(',')[0].strip().strip('"')
        if id in prods:
                review = decode(line.strip())
                d = pd.DataFrame({"product.id": [review['asin']],
                                  "rating": [review['overall']],
                                  "review": [review['reviewText']]})
                df = df.append(d, ignore_index=True)
    return df

Now filter reviews by the produts of interest found in Task 4.

In [17]:
reviews = filter_reviews(products)
reviews

,product.id,rating,review
0,B000BFWJTO,5.0,i like that pen!!
1,B000BFWJTO,5.0,Super handy for a traveling salesman! All I ne...
2,B000BFWJTO,5.0,Since going to the electronic flight bag (iPad...
3,B000BFWJTO,5.0,As advertised no complaints. This is the 2nd o...
4,B000BFWJTO,4.0,"This is a great pen as many have said, and as ..."
5,B000BFWJTO,5.0,i Always knew cross will go beyond my expectat...
6,B000BFWJTO,4.0,"Works great, worth every penny spent on it, th..."
7,B000BFWJTO,1.0,Will not refer to any one!Product is useless a...
8,B000BFWJTO,4.0,Everything is great but the lead is very britt...
9,B000BFWJTO,5.0,What an awesome writing too! All of the diffe...


**Task 7:** Summarize the reviews into a data frame called `review_summaries` with three columns: `product.id`, `count` for the number of reviews, and `ave.rating` for the mean rating.

In [18]:
review_summaries=reviews.groupby('product.id').aggregate({'review':lambda x: len(x), 'rating':np.mean})
review_summaries.columns = ['count', 'ave.rating'] 
review_summaries.head()

,count,ave.rating
product.id,,
B00002EQCF,10,3.800000
B000BFWJTO,209,4.377990
B000F4CUKY,28,3.214286
B0013T1Z7O,4,5.000000
B001CTETMI,3,2.000000


**Task 8:** Merge `review_summaries` with `products` on the `product.id` variable.

In [19]:
mergeproducts=pd.merge(review_summaries, products, on='product.id')
mergeproducts.head()

,product.id,count,ave.rating,price
0,B00002EQCF,10,3.800000,99.0
1,B000BFWJTO,209,4.377990,33.89
2,B000F4CUKY,28,3.214286,117.57
3,B0013T1Z7O,4,5.000000,59.99
4,B001CTETMI,3,2.000000,189.99


**Task 10:** Write function `filter_products_by_id` that takes in a product identifier argument `idx` and returns returns the matching product information from `/data/Assignment-2/products.json`.

In [20]:
def filter_products_by_id(idx):
   
    regex1 = re.compile(r"'asin': '(.+?)',")
    regex2 = re.compile(r"'title': '(.+?)',")
    pid=[regex1.findall(t) for t in products_raw]
    title=[regex2.findall(t) for t in products_raw]
    df=pd.DataFrame({"product.id":pid,"title":title})
    result1=df.applymap(lambda x: x[0] if len(x)>0 else np.nan)
   
    merge=pd.merge(mergeproducts, result1, on='product.id')
    result=pd.merge(merge, idx, on='product.id',how='inner')
    
    
   
    return (result)

**Task 11:** Using the plot form Task 9, the `filter_products_by_id` function, and the `review_summaries` data frame, identify 3 competitor products and extract the relevant information required for the assignment write-up.

In [21]:
finallist=[['B000BFWJTO'], ['B000F4CUKY'], ['B003LVKQ18']]
finalproducts=pd.DataFrame(finallist,columns=['product.id'])
finalproducts

,product.id
0,B000BFWJTO
1,B000F4CUKY
2,B003LVKQ18


In [23]:
filter_products_by_id(finalproducts)

,product.id,count,ave.rating,price,title
0,B000BFWJTO,209,4.377990,33.89,Tech 3 Multi-function Pen with Chrome Accents
1,B000F4CUKY,28,3.214286,117.57,Logitech IO2 Digital Pen ( 965154-0403 )
2,B003LVKQ18,109,4.449541,59.99,Wacom CTL460 Bamboo Pen Tablet (Factory Refurb...


**Task 12:** Filter the `reviews` data frame for each of the products so that we may read the corresponding reviews.

In [22]:
finalreviews = filter_reviews(finalproducts)